In [ ]:
import pandas as pd
import datetime as datetime

In [ ]:
report_table = pd.read_csv("opp_table_3.csv")
SFDC_table = pd.read_csv("SFDC.csv")
emp_table = pd.read_csv("emp_tbl.csv")


In [ ]:
print(report_table.columns.tolist())

In [ ]:
print(SFDC_table.columns.tolist())

Total Opportunities in Current BD Year

In [ ]:
report_table_bd = report_table[report_table["CLOSE_DATE"] >= "2023-10-01"]
report_table_bd = report_table_bd[report_table_bd["CREATED_DATE"] <= "2023-12-13"]

In [ ]:
report_table_bd

In [ ]:
SFDC_table = SFDC_table.drop(SFDC_table[SFDC_table["Opportunity ID"] == "006RQ0000040H1o"].index)

In [ ]:
SFDC_table

In [ ]:
# Check if there are different IDs
list1 = report_table_bd["OPPORTUNITYID"]
list2 = SFDC_table["Opportunity ID"]

print(list(set(list1) - set(list2)))
print(list(set(list1) - set(list2)))

Check Annual Agreement Value (Net)

In [ ]:
report_table_bd["TOTAL_ANNUAL_AGREEMENT_NET"].sum()

In [ ]:
SFDC_table["Annual Agreement Value (Net)"].sum()

In [ ]:
# Investigate Difference
report_small = report_table_bd[["OPPORTUNITYID", "TOTAL_ANNUAL_AGREEMENT_NET"]]
sfdc_small = SFDC_table[["Opportunity ID", "Annual Agreement Value (Net)"]]
                    

In [ ]:
report_small_join = report_small.merge(sfdc_small, how = "left", left_on = "OPPORTUNITYID", right_on = "Opportunity ID")
report_small_join.drop("Opportunity ID", axis = 1, inplace = True)

In [ ]:
report_small_join

In [ ]:
report_small_join["match"] = report_small_join["TOTAL_ANNUAL_AGREEMENT_NET"] == report_small_join["Annual Agreement Value (Net)"]

In [ ]:
report_small_join[report_small_join["match"] == 0]

In [ ]:
report_small_join[report_small_join["match"] == 0]["OPPORTUNITYID"].tolist()

Check Expected Revenue  (which is Total Annual Agreement Value (Gross) * Avg Close Rate)

In [ ]:
#Get Total Annual Agreement Value for both first
report_table_tot = report_table_bd[report_table_bd["CREATED_DATE"] >= "06-23-2022"]
report_table_tot = report_table_tot[report_table_tot["RECORD_TYPE_NAME"] == "Advisory"]
report_table_tot = report_table_tot[report_table_tot["SEGMENT"] != "Open"]
report_table_tot = report_table_tot[report_table_tot["CLOSE_DATE"] >= "06-23-2022"]

report_table_TW = report_table_tot[report_table_tot["STAGE_NAME"] == "Stage 4 - New Client"]

In [ ]:
report_table_TW.shape[0]

In [ ]:
report_table_TC = report_table_tot[report_table_tot["STAGE_NAME"].isin(["Stage 4 - New Client", "Fallout"])]


In [ ]:
report_table_TC.shape[0]

In [ ]:
SFDC_table_tot = SFDC_table[SFDC_table["Created Date"] >= "06-23-2022"]
SFDC_table_tot  = SFDC_table_tot [SFDC_table_tot ["Opportunity Record Type"] == "Advisory"]
SFDC_table_tot  = SFDC_table_tot [SFDC_table_tot ["Segment"] != "Open"]
SFDC_table_tot  = SFDC_table_tot [SFDC_table_tot ["Close Date"] >= "06-23-2022"]

SFDC_table_TW  = SFDC_table_tot[SFDC_table_tot["Stage"] == "Stage 4 - New Client"]

In [ ]:
SFDC_table_TW.shape[0]

In [ ]:
SFDC_table_TC = SFDC_table_tot[SFDC_table_tot["Stage"].isin(["Stage 4 - New Client", "Fallout"])]


In [ ]:
SFDC_table_TC.shape[0]

In [ ]:
report_table_tot["TOTAL_ANNUAL_AGREEMENT_NET"].sum() * (report_table_TW.shape[0]/report_table_TC.shape[0])

In [ ]:
SFDC_table_tot["Annual Agreement Value (Net)"].sum() * (SFDC_table_TW.shape[0]/SFDC_table_TC.shape[0])

Compare # of Won & Closed Opportunities

In [ ]:
SFDC_table

Check Won

In [ ]:
reporting_won = report_table_bd[report_table_bd["IS_WON"] == 1]
reporting_lost = report_table_bd[report_table_bd["IS_WON"] == 0]

sfdc_won = SFDC_table[SFDC_table["Won"] == 1]
sfdc_lost = SFDC_table[SFDC_table["Won"] == 0]

In [ ]:
SFDC_table["Won"].value_counts()

In [ ]:
report_table_bd["IS_WON"].value_counts()

In [ ]:
wonlost_join = sfdc_won.merge(report_table_bd, how = "left", right_on = "OPPORTUNITYID", left_on = "Opportunity ID")

In [ ]:
wonlost_join = wonlost_join[["OPPORTUNITYID", "IS_WON", "Won"]]

In [ ]:
wonlost_join["match"] = wonlost_join["IS_WON"] == wonlost_join["Won"]

In [ ]:
wonlost_join[wonlost_join["match"] == False]

Check Closed Opps

In [ ]:
reporting_close = report_table_bd[report_table_bd["IS_CLOSED"] == 1]
reporting_open = report_table_bd[report_table_bd["IS_CLOSED"] == 1]

sfdc_close = SFDC_table[SFDC_table["Closed"] == 1]
sfdc_open = SFDC_table[SFDC_table["Closed"] == 0]

In [ ]:
SFDC_table["Closed"].value_counts()

In [ ]:
report_table_bd["IS_CLOSED"].value_counts()

In [ ]:
closeopen_join = sfdc_close.merge(report_table_bd, how = "left", right_on = "OPPORTUNITYID", left_on = "Opportunity ID")
closeopen_join = closeopen_join[["OPPORTUNITYID", "IS_CLOSED", "Closed"]]

In [ ]:
closeopen_join["match"] = closeopen_join["IS_CLOSED"] == closeopen_join["Closed"]

In [ ]:
closeopen_join[closeopen_join["match"] == False]

Check Opportunity Owners Match

In [ ]:
report_opp_owner = report_table_bd[["OPPORTUNITYID", "OPPORTUNITY_OWNER"]]

In [ ]:
report_opp_owner.shape[0]

In [ ]:
sfdc_opp_owner = SFDC_table[["Opportunity ID", "Opportunity Owner"]]

In [ ]:
sfdc_opp_owner.shape[0]

In [ ]:
opp_join = report_opp_owner.merge(sfdc_opp_owner, how = "inner", left_on = "OPPORTUNITYID", right_on = "Opportunity ID")
opp_join.drop("Opportunity ID", axis = 1, inplace = True)

In [ ]:
opp_join = opp_join.merge(emp_table, how = "left", left_on = "OPPORTUNITY_OWNER", right_on = "USER_ID")
opp_join.drop("USER_ID", axis = 1, inplace = True)

In [ ]:
opp_join["match"] = opp_join["Opportunity Owner"] == opp_join["EMPLOYEE_NAME"]

In [ ]:
opp_join[opp_join["match"] == 0]

Check Lead Advisors Match

In [ ]:
report_la = report_table_bd[["OPPORTUNITYID", "LEAD_ADVISOR_NAME"]]

In [ ]:
sfdc_la = SFDC_table[["Opportunity ID", "Lead Advisor"]]

In [ ]:
la_join = report_la.merge(sfdc_la, how = "inner", left_on = "OPPORTUNITYID", right_on = "Opportunity ID")
la_join.drop("Opportunity ID", axis = 1, inplace = True)


In [ ]:
la_join

In [ ]:
la_join["match"] = la_join["LEAD_ADVISOR_NAME"] == la_join["Lead Advisor"]

In [ ]:
match_df = la_join[la_join["match"] == 0]

In [ ]:
match_df[match_df["Lead Advisor"] == "Laura Hearn"]["OPPORTUNITYID"]

Lead Source Matching


In [ ]:
report_ls = report_table_bd[["OPPORTUNITYID", "LEAD_SOURCE"]]

In [ ]:
sfdc_ls = SFDC_table[["Opportunity ID", "Lead Source"]]

In [ ]:
ls_join = report_ls.merge(sfdc_ls, how = "inner", left_on = "OPPORTUNITYID", right_on = "Opportunity ID")
ls_join.drop("Opportunity ID", axis = 1, inplace = True)

In [ ]:
ls_join

In [ ]:
ls_join["match"] = ls_join["LEAD_SOURCE"] == ls_join["Lead Source"]

In [ ]:
match_ls_df = ls_join[ls_join["match"] == 0]

In [ ]:
match_ls_df

Within Revenue Channels, check closed business amount is correct

In [ ]:
report_table_bd

In [ ]:
report_table_bd.groupby(["LEAD_SOURCE"])["TOTAL_ANNUAL_AGREEMENT_NET"].sum()

In [ ]:
# Check out Slef Gen
report_selfgen = report_table_bd[report_table_bd["LEAD_SOURCE"] == "Self Generated"]

In [ ]:
report_selfgen

In [ ]:
report_selfgen_small = report_selfgen[["OPPORTUNITYID", "TOTAL_ANNUAL_AGREEMENT_NET"]]

In [ ]:
report_selfgen_small

In [ ]:
# Get SFDC Counterpart
sfdc_selfgen_small = SFDC_table[["Opportunity ID", "Lead Source", "Annual Agreement Value (Net)"]]

In [ ]:
sfdc_selfgen_small = sfdc_selfgen_small[sfdc_selfgen_small["Lead Source"] == "Self Generated"]
sfdc_selfgen_small.drop("Lead Source", axis = 1, inplace = True)

In [ ]:
sfdc_selfgen_small

In [ ]:
selfgen_join = report_selfgen_small.merge(sfdc_selfgen_small, how = "left", left_on = "OPPORTUNITYID", right_on = "Opportunity ID")

In [ ]:
selfgen_join

In [ ]:
selfgen_join["match"] = selfgen_join["TOTAL_ANNUAL_AGREEMENT_NET"] == selfgen_join["Annual Agreement Value (Net)"]

In [ ]:
selfgen_join

In [ ]:
selfgen_join[selfgen_join["match"] == 0]

By Lead Advisor, Check Amt of Closed Business is correct

In [ ]:
report_table_bd.groupby(["LEAD_ADVISOR_NAME"])["TOTAL_ANNUAL_AGREEMENT_NET"].sum()

In [ ]:
report_bb = report_table_bd[report_table_bd["LEAD_ADVISOR_NAME"] == "Brian Burgess"][["OPPORTUNITYID", "TOTAL_ANNUAL_AGREEMENT_NET"]]
report_jf = report_table_bd[report_table_bd["LEAD_ADVISOR_NAME"] == "Jennifer Facini"][["OPPORTUNITYID", "TOTAL_ANNUAL_AGREEMENT_NET"]]
report_jr = report_table_bd[report_table_bd["LEAD_ADVISOR_NAME"] == "Jesica Ray"][["OPPORTUNITYID", "TOTAL_ANNUAL_AGREEMENT_NET"]]
report_mm = report_table_bd[report_table_bd["LEAD_ADVISOR_NAME"] == "Matt Mormino"][["OPPORTUNITYID", "TOTAL_ANNUAL_AGREEMENT_NET"]]

In [ ]:
# Get SFDC Counterpart
sfdc_bb_small = SFDC_table[["Opportunity ID", "Lead Advisor", "Annual Agreement Value (Net)"]]
sfdc_bb_small = sfdc_bb_small[sfdc_bb_small["Lead Advisor"] == "Brian Burgess"]
sfdc_bb_small.drop("Lead Advisor", axis = 1, inplace = True)

sfdc_jf_small = SFDC_table[["Opportunity ID", "Lead Advisor", "Annual Agreement Value (Net)"]]
sfdc_jf_small = sfdc_jf_small[sfdc_jf_small["Lead Advisor"] == "Jennifer Facini"]
sfdc_jf_small.drop("Lead Advisor", axis = 1, inplace = True)

sfdc_jr_small = SFDC_table[["Opportunity ID", "Lead Advisor", "Annual Agreement Value (Net)"]]
sfdc_jr_small = sfdc_jr_small[sfdc_jr_small["Lead Advisor"] == "Jesica Ray"]
sfdc_jr_small.drop("Lead Advisor", axis = 1, inplace = True)

sfdc_mm_small = SFDC_table[["Opportunity ID", "Lead Advisor", "Annual Agreement Value (Net)"]]
sfdc_mm_small = sfdc_mm_small[sfdc_mm_small["Lead Advisor"] == "Matt Mormino"]
sfdc_mm_small.drop("Lead Advisor", axis = 1, inplace = True)

In [ ]:
bb_join = report_bb.merge(sfdc_bb_small, how = "left", left_on = "OPPORTUNITYID", right_on = "Opportunity ID")
jf_join = report_jf.merge(sfdc_jf_small, how = "left", left_on = "OPPORTUNITYID", right_on = "Opportunity ID")
jr_join = report_jr.merge(sfdc_jr_small, how = "left", left_on = "OPPORTUNITYID", right_on = "Opportunity ID")
mm_join = sfdc_mm_small.merge(report_mm, how = "left", left_on = "Opportunity ID", right_on = "OPPORTUNITYID")

In [ ]:
bb_join["match"] = bb_join["TOTAL_ANNUAL_AGREEMENT_NET"] == bb_join["Annual Agreement Value (Net)"]
jf_join["match"] = jf_join["TOTAL_ANNUAL_AGREEMENT_NET"] == jf_join["Annual Agreement Value (Net)"]
jr_join["match"] = jr_join["TOTAL_ANNUAL_AGREEMENT_NET"] == jr_join["Annual Agreement Value (Net)"]
mm_join["match"] = mm_join["TOTAL_ANNUAL_AGREEMENT_NET"] == mm_join["Annual Agreement Value (Net)"]

In [ ]:
bb_join[bb_join["match"] == 0]

In [ ]:
jf_join[jf_join["match"] == 0]

In [ ]:
jr_join[jr_join["match"] == 0]

In [ ]:
mm_join[mm_join["match"] == 0]